# MERFISH whole brain spatial transcriptomics (part 2a)

In part 1, we explored two examples looking at the expression of canonical neurotransmitter transporter genes and gene Tac2 in the one coronal section. In this notebook, we will prepare data so that we can repeat the examples for all cells spanning the whole brain. This notebook takes ~20 seconds to run.

The results from this notebook has already been cached and saved. As such, if needed you can skip this notebook and continue with part 2b.

You need to be connected to the internet to run this notebook and that you have downloaded the all the log2 expression matrices associated with MERFISH-C57BL6J-638850 dataset.

In [1]:
import os
import pandas as pd
import numpy as np
import anndata
import time
import json
import requests

The prerequisite for running this notebook is that the data have been downloaded to local directory maintaining the organization from the manifest.json. **Change the download_base variable to where you have downloaded the data in your system.**

In [2]:
download_base = '../../abc_download_root'

url = 'https://allen-brain-cell-atlas.s3-us-west-2.amazonaws.com/releases/20230630/manifest.json'
manifest = json.loads(requests.get(url).text)
    
metadata = manifest['file_listing']['MERFISH-C57BL6J-638850']['metadata']

In [3]:
view_directory = os.path.join( download_base, 
                               manifest['directory_listing']['MERFISH-C57BL6J-638850']['directories']['metadata']['relative_path'], 
                              'views')
cache_views = False
if cache_views :
    os.makedirs( view_directory, exist_ok=True )

In [4]:
rpath = metadata['cell_metadata']['files']['csv']['relative_path']
file = os.path.join( download_base, rpath)
cell = pd.read_csv(file, dtype={'cell_label':str})
cell.set_index('cell_label',inplace=True)
print(len(cell))

4330907


In [5]:
matrices = cell.groupby('matrix_label')[['brain_section_label']].count()
matrices

,brain_section_label
matrix_label,
C57BL6J-638850.01,26217
C57BL6J-638850.02,29286
C57BL6J-638850.03,36028
C57BL6J-638850.04,47445
C57BL6J-638850.05,50990
C57BL6J-638850.06,50883
C57BL6J-638850.08,51941
C57BL6J-638850.09,75870
C57BL6J-638850.10,50248


In [6]:
expression_matrices = manifest['file_listing']['MERFISH-C57BL6J-638850']['expression_matrices']

In [7]:
matrix_label = matrices.index[0]
rpath = expression_matrices[matrix_label]['log2']['files']['h5ad']['relative_path']
file = os.path.join( download_base, rpath)
print(file)

../../abc_download_root/expression_matrices/MERFISH-C57BL6J-638850/20230630/C57BL6J-638850.01-log2.h5ad


In [8]:
ad = anndata.read_h5ad(file,backed='r')
gene = ad.var

In [9]:
ntgenes = ['Slc17a7','Slc17a6','Slc17a8','Slc32a1','Slc6a5','Slc18a3','Slc6a3','Slc6a4','Slc6a2']
exgenes = ['Tac2']
gnames = ntgenes + exgenes
pred = [x in gnames for x in gene.gene_symbol]
gene_filtered = gene[pred]
gene_filtered

,gene_symbol,transcript_identifier
gene_identifier,,
ENSMUSG00000030500,Slc17a6,ENSMUST00000032710
ENSMUSG00000037771,Slc32a1,ENSMUST00000045738
ENSMUSG00000025400,Tac2,ENSMUST00000026466
ENSMUSG00000039728,Slc6a5,ENSMUST00000056442
ENSMUSG00000070570,Slc17a7,ENSMUST00000085374
ENSMUSG00000019935,Slc17a8,ENSMUST00000020102
ENSMUSG00000021609,Slc6a3,ENSMUST00000022100
ENSMUSG00000020838,Slc6a4,ENSMUST00000021195


In [10]:
# create empty gene expression dataframe
gdata = pd.DataFrame(index=cell.index,columns=gene_filtered.index)
count = 0
total_start = time.process_time()

for mp in matrices.index :
    
    print(mp)
    
    rpath = expression_matrices[mp]['log2']['files']['h5ad']['relative_path']
    file = os.path.join( download_base, rpath)
        
    start = time.process_time()
    ad = anndata.read_h5ad(file,backed='r')
    exp = ad[:,gene_filtered.index].to_df()
    gdata.loc[ exp.index, gene_filtered.index ] = exp
    print(" - time taken: ", time.process_time() - start)
    
    ad.file.close()
    del ad
    
    count += 1
    
    #if count > 2 :
    #    break
        
print("total time taken: ", time.process_time() - total_start)
    

C57BL6J-638850.01
 - time taken:  2.07290858
C57BL6J-638850.02
 - time taken:  0.10913115299999987
C57BL6J-638850.03
 - time taken:  0.12849889599999997
C57BL6J-638850.04
 - time taken:  0.1750259370000009
C57BL6J-638850.05
 - time taken:  0.19173907899999953
C57BL6J-638850.06
 - time taken:  0.18017767599999956
C57BL6J-638850.08
 - time taken:  0.1913277599999983
C57BL6J-638850.09
 - time taken:  0.2148004640000014
C57BL6J-638850.10
 - time taken:  0.15845831000000032
C57BL6J-638850.11
 - time taken:  0.17424620400000101
C57BL6J-638850.12
 - time taken:  0.21298487400000177
C57BL6J-638850.13
 - time taken:  0.26169406699999875
C57BL6J-638850.14
 - time taken:  0.3134420270000007
C57BL6J-638850.15
 - time taken:  0.3036803419999998
C57BL6J-638850.16
 - time taken:  0.34225327499999914
C57BL6J-638850.17
 - time taken:  0.24623338900000036
C57BL6J-638850.18
 - time taken:  0.19589953699999896
C57BL6J-638850.19
 - time taken:  0.2607699640000014
C57BL6J-638850.24
 - time taken:  0.2614625

In [11]:
# change columns from index to gene symbol
gdata.columns = gene_filtered.gene_symbol
pred = pd.notna(gdata[gdata.columns[0]])
gdata = gdata[pred].copy(deep=True)
print(len(gdata))

4330907


In [12]:
if cache_views :
    file = os.path.join( view_directory, 'example_genes_all_cells_expression.csv')
    gdata.to_csv( file )